In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Carregar o DataFrame com os dados dos filmes
df = pd.read_csv('dados.csv')

# Pré-processamento dos dados
# Combine título e sinopse em uma única representação de texto
df['combined_features'] = df['Titulo'] + ' ' + df['Sinopse']

# Codificar os títulos dos filmes para valores numéricos
label_encoder = LabelEncoder()
df['Titulo_encoded'] = label_encoder.fit_transform(df['Titulo'])

# Vetorizar as características combinadas
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['combined_features'])
y = df['Titulo_encoded']  # Usar os títulos codificados como classes

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obter títulos únicos apenas do conjunto de treinamento
unique_titles_train = df.loc[y_train.index, 'Titulo'].unique()

# Filtrar o conjunto de teste para incluir apenas os títulos presentes no conjunto de treinamento
X_test_filtered = X_test[y_test.isin(unique_titles_train)]
y_test_filtered = y_test[y_test.isin(unique_titles_train)]

# Definir os parâmetros do modelo XGBoost
params = {
    'objective': 'multi:softmax',  # Usar a função de perda multi-softmax para classificação multiclasse
    'num_class': len(np.unique(y_train)),  # Número de classes é o número de títulos únicos no conjunto de treinamento
    'random_state': 42
}

# Treinar o modelo
model_xgb = xgb.XGBClassifier(**params)
model_xgb.fit(X_train, y_train)

# Fazer a previsão usando o modelo
entrada_usuario = input("Digite suas preferências: ")
entrada_usuario_vector = vectorizer.transform([entrada_usuario])

# Prever o índice do filme recomendado
indice_filme_recomendado = model_xgb.predict(entrada_usuario_vector)[0]

# Decodificar o índice para obter o título do filme recomendado
filme_recomendado = label_encoder.inverse_transform([indice_filme_recomendado])[0]

# Imprimir o filme recomendado
print("Filme recomendado:", filme_recomendado)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 4547 4548 4549], got [   0    1    2 ... 5685 5686 5687]